In [ ]:
!pip install datasets
!pip install transformers
!pip install sentencepiece
!pip install evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 29.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 24.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 18.4 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 70.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 76.6 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-whe

In [ ]:
from datasets import load_dataset
from transformers import T5Tokenizer, T5ForConditionalGeneration
import torch
import numpy as np

PART 4

In [ ]:
from transformers import AutoTokenizer, AutoModel, DataCollatorWithPadding, AutoModelForSequenceClassification
from transformers import TrainingArguments
from transformers import Trainer
import evaluate

In [ ]:
french_dataset = load_dataset("pranjali97/french_translated_snli")

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating validation split:   0%|          | 0/10000 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/550152 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/10000 [00:00<?, ? examples/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/pranjali97___parquet/pranjali97--french_translated_snli-e3cf0920dd590347/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
french_dataset

DatasetDict({
    validation: Dataset({
        features: ['premise', 'hypothesis', 'label', 'translated_premise', 'translated_hypothesis'],
        num_rows: 10000
    })
    train: Dataset({
        features: ['premise', 'hypothesis', 'label', 'translated_premise', 'translated_hypothesis'],
        num_rows: 550152
    })
    test: Dataset({
        features: ['premise', 'hypothesis', 'label', 'translated_premise', 'translated_hypothesis'],
        num_rows: 10000
    })
})

Removing class label -1

In [ ]:
french_dataset = french_dataset.filter(lambda example: example['label'] in [0,1,2])

Filter:   0%|          | 0/10000 [00:00<?, ? examples/s]

Filter:   0%|          | 0/550152 [00:00<?, ? examples/s]

Filter:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [ ]:
def merge_hypo_premise(example):
  example['text'] = example['translated_premise'] + example['translated_hypothesis']
  return example

In [ ]:
french_dataset = french_dataset.map(merge_hypo_premise)

Map:   0%|          | 0/9842 [00:00<?, ? examples/s]

Map:   0%|          | 0/549367 [00:00<?, ? examples/s]

Map:   0%|          | 0/9824 [00:00<?, ? examples/s]

In [ ]:
french_dataset

DatasetDict({
    validation: Dataset({
        features: ['premise', 'hypothesis', 'label', 'translated_premise', 'translated_hypothesis', 'text'],
        num_rows: 9842
    })
    train: Dataset({
        features: ['premise', 'hypothesis', 'label', 'translated_premise', 'translated_hypothesis', 'text'],
        num_rows: 549367
    })
    test: Dataset({
        features: ['premise', 'hypothesis', 'label', 'translated_premise', 'translated_hypothesis', 'text'],
        num_rows: 9824
    })
})

In [ ]:
train_data = french_dataset['train']
validation_data = french_dataset['validation']
test_data = french_dataset['test']

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("cmarkea/distilcamembert-base")

In [ ]:
tokenized_train = train_data.map(lambda example: tokenizer(example['text'], truncation=True), batched=True, batch_size=256)
tokenized_eval = validation_data.map(lambda example: tokenizer(example['text'], truncation=True), batched=True, batch_size=256)
tokenized_test = test_data.map(lambda example: tokenizer(example['text'], truncation=True), batched=True, batch_size=256)

Map:   0%|          | 0/549367 [00:00<?, ? examples/s]

Map:   0%|          | 0/9842 [00:00<?, ? examples/s]

Map:   0%|          | 0/9824 [00:00<?, ? examples/s]

In [ ]:
tokenized_train

Dataset({
    features: ['premise', 'hypothesis', 'label', 'translated_premise', 'translated_hypothesis', 'text', 'input_ids', 'attention_mask'],
    num_rows: 549367
})

In [ ]:
tokenized_train.set_format(type="pt", columns=['input_ids', 'attention_mask', 'label'])


In [ ]:
model = AutoModelForSequenceClassification.from_pretrained("cmarkea/distilcamembert-base", num_labels = 3)

Some weights of the model checkpoint at cmarkea/distilcamembert-base were not used when initializing CamembertForSequenceClassification: ['lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.bias', 'lm_head.decoder.weight', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing CamembertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing CamembertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of CamembertForSequenceClassification were not initialized from the model checkpoint at cmarkea/distilcamembert-base and are newly initialized: ['classifier.out_proj.bias', 'clas

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

In [ ]:
training_args: TrainingArguments = TrainingArguments(
        output_dir="part4_training",
        do_train=True,
        do_eval=True,
        do_predict=True,
        evaluation_strategy="steps",
        eval_steps=128,
        per_device_train_batch_size=128,
        per_device_eval_batch_size=128,
        save_steps=128,
        save_strategy="steps",
        save_total_limit=5,
        num_train_epochs=5,
        metric_for_best_model="accuracy",
        load_best_model_at_end=True,
        dataloader_num_workers=0,  # set to 0 when debugging and >1 when running!
    )

In [ ]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
f1_metric = evaluate.load('f1')
acc_metric = evaluate.load('accuracy')

def my_compute_metrics(eval_pred):
  logits, labels = eval_pred.predictions, eval_pred.label_ids
  predictions = np.argmax(logits, axis=1)
  result = {}
  result.update(acc_metric.compute(predictions=predictions, references=labels))
  result.update(f1_metric.compute(predictions=predictions, references=labels, average = "macro"))
  return result


In [ ]:
trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_eval,
    tokenizer=tokenizer,
    compute_metrics=my_compute_metrics,
)

In [ ]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `CamembertForSequenceClassification.forward` and have been ignored: translated_premise, translated_hypothesis, text, premise, hypothesis. If translated_premise, translated_hypothesis, text, premise, hypothesis are not expected by `CamembertForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 549367
  Num Epochs = 5
  Instantaneous batch size per device = 128
  Total train batch size (w. parallel, distributed & accumulation) = 128
  Gradient Accumulation steps = 1
  Total optimization steps = 21460
  Number of trainable parameters = 6809702

Step,Training Loss,Validation Loss,Accuracy,F1
128,No log,0.792199,0.651900,0.648187
256,No log,0.725543,0.689596,0.688883
384,No log,0.689994,0.705344,0.704418
512,0.804400,0.691610,0.708596,0.706526
640,0.804400,0.657536,0.721805,0.722498
768,0.804400,0.655850,0.727698,0.726279
896,0.804400,0.648682,0.732981,0.732264
1024,0.697600,0.642432,0.734302,0.733651
1152,0.697600,0.621793,0.742634,0.741945
1280,0.697600,0.618390,0.744463,0.742440


The following columns in the evaluation set don't have a corresponding argument in `CamembertForSequenceClassification.forward` and have been ignored: translated_premise, translated_hypothesis, text, premise, hypothesis. If translated_premise, translated_hypothesis, text, premise, hypothesis are not expected by `CamembertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 9842
  Batch size = 128
Saving model checkpoint to part4_training/checkpoint-128
Configuration saved in part4_training/checkpoint-128/config.json
Model weights saved in part4_training/checkpoint-128/pytorch_model.bin
tokenizer config file saved in part4_training/checkpoint-128/tokenizer_config.json
Special tokens file saved in part4_training/checkpoint-128/special_tokens_map.json
The following columns in the evaluation set don't have a corresponding argument in `CamembertForSequenceClassification.forward` and have been ignored: translated_premise, tr

TrainOutput(global_step=21460, training_loss=0.5082301058951428, metrics={'train_runtime': 9647.8594, 'train_samples_per_second': 284.709, 'train_steps_per_second': 2.224, 'total_flos': 2.379060091224292e+16, 'train_loss': 0.5082301058951428, 'epoch': 5.0})

In [ ]:
trainer.evaluate(metric_key_prefix="test", eval_dataset=tokenized_test)

The following columns in the evaluation set don't have a corresponding argument in `CamembertForSequenceClassification.forward` and have been ignored: translated_premise, translated_hypothesis, text, premise, hypothesis. If translated_premise, translated_hypothesis, text, premise, hypothesis are not expected by `CamembertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 9824
  Batch size = 128


{'test_loss': 0.5498430728912354,
 'test_accuracy': 0.7898004885993485,
 'test_f1': 0.7898816349771822,
 'test_runtime': 9.2157,
 'test_samples_per_second': 1066.012,
 'test_steps_per_second': 8.355,
 'epoch': 5.0}